In [ ]:
# Setup
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, cohen_kappa_score
import google.generativeai as genai
from openai import OpenAI
from google.colab import files

Llama_KEY = input("Paste your Llama API key: ").strip()
GEMINI_KEY = input("Paste your Gemini API key: ").strip()

os.environ["Llama_API_KEY"] = Llama_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_KEY

# Initialize clients
llama_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["Llama_API_KEY"]
)
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
# 1. Load dataset
def load_dataset(file_path):
    with open(file_path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    return pd.DataFrame({"raw_line": lines})

In [ ]:
# 2. Preprocessing
def preprocess_text(df, text_column="raw_line"):
    speaker_turns = []
    for line in df[text_column]:
        line = str(line).strip()
        if ":" in line:
            speaker, utterance = line.split(":", 1)
            speaker_turns.append({"speaker": speaker.strip(), "utterance": utterance.strip()})
        else:
            speaker_turns.append({"speaker": "Unknown", "utterance": line})
    return pd.DataFrame(speaker_turns)

In [ ]:
# Trait information
TRAIT_INFO = {
    "Expressive": "The speaker is open, shows a range of emotions, and readily shares thoughts and feelings.",
    "Reserved": "The speaker is cautious, formal, and holds back on personal details.",
    "Assertive": "The speaker is confident, direct, and takes the lead in the conversation.",
    "Supportive": "The speaker is empathetic, agreeable, and focuses on building rapport.",
    "Analytical": "The speaker is logical, fact-oriented, and structured in their responses.",
    "Spontaneous": "The speaker is unpredictable, energetic, and frequently changes the topic or tone.",
    "Neutral": "The speaker's trait is unclear or does not fit any of the other categories."
}

# 3. Human annotation loop (chunked)
def human_annotation(df, rounds_per_chunk=1):
    print("Trait explanations (refer to this before annotating):\n")
    for trait, explanation in TRAIT_INFO.items():
        print(f"{trait}: {explanation}")
    print("\n--- Starting annotation ---\n")

    # Each round = 2 turns (Human1 + Human2)
    chunk_size = rounds_per_chunk * 2
    chunks = [df.iloc[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
    all_annotations = []

    for idx, chunk in enumerate(chunks):
        print(f"\n--- Chunk {idx+1} ---\n")
        print("Chunk content:")
        for _, row in chunk.iterrows():
            print(f"{row['speaker']}: {row['utterance']}")

        chunk_annotations = {}
        participants = chunk['speaker'].unique()
        for participant in participants:
            participant_turns = chunk[chunk['speaker'] == participant]['utterance'].tolist()
            print(f"\nParticipant: {participant}")
            print("Their turns in this chunk:")
            for turn in participant_turns:
                print(f"- {turn}")

            print("Available traits: " + " | ".join(TRAIT_INFO.keys()))
            choice = input("Enter trait for this participant in this chunk: ").strip().title()
            if choice not in TRAIT_INFO:
                choice = "Neutral"
            chunk_annotations[participant] = choice
        all_annotations.append(chunk_annotations)

        next_chunk = input("Move to next chunk? (y/n): ").strip().lower()
        if next_chunk != "y":
            break

    return all_annotations, chunks


# Aggregate traits + distribution
def aggregate_traits(chunk_annotations):
  trait_counts = {}
  for chunk in chunk_annotations:
      for participant, trait in chunk.items():
          if participant not in trait_counts:
              trait_counts[participant] = {}
          if trait not in trait_counts[participant]:
              trait_counts[participant][trait] = 0
          trait_counts[participant][trait] += 1

  overall_traits = {}
  for participant, counts in trait_counts.items():
      overall_traits[participant] = max(counts, key=counts.get)

  return overall_traits, trait_counts

In [ ]:
# Shared prompt builder
def build_prompt(chunk):
    chunk_text = "\n".join([f"{row['speaker']}: {row['utterance']}" for _, row in chunk.iterrows()])
    trait_list = "\n".join([f"{name}: {explanation}" for name, explanation in TRAIT_INFO.items()])
    prompt = f"""
Analyze this conversation chunk and determine the conversational trait of each participant.
Use only ONE trait per participant from the list below:

{trait_list}

Conversation chunk:
{chunk_text}

Respond with the traits as a dictionary, no further explanations, e.g.:
{{"Human 1": "Expressive", "Human 2": "Analytical"}}
"""
    return prompt


# Model annotation chunked
def model_annotation(chunk, model_name="llama"):
    prompt = build_prompt(chunk)

    if model_name == "llama":
        response = llama_client.chat.completions.create(
            model="meta-llama/llama-4-maverick:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        text = response.choices[0].message.content.strip()

    elif model_name == "gemini":
        model = genai.GenerativeModel("gemini-2.5-flash-lite")
        response = model.generate_content(prompt)
        text = response.text.strip()
    else:
        raise ValueError("Unknown model")

    # Convert model response to dict
    try:
        labels_dict = eval(text)
    except:
        labels_dict = {participant: "Neutral" for participant in chunk['speaker'].unique()}

    return labels_dict

In [ ]:
# 5. Evaluation + Plot
def evaluate_model(human_chunks, model_chunks, model_name):
    human_flat = []
    model_flat = []

    for h_chunk, m_chunk in zip(human_chunks, model_chunks):
        for participant in h_chunk.keys():
            human_flat.append(h_chunk[participant])
            model_flat.append(m_chunk.get(participant, "Neutral"))

    acc = accuracy_score(human_flat, model_flat)
    kappa = cohen_kappa_score(human_flat, model_flat)
    print(f"{model_name} - Accuracy: {acc:.2f}, Kappa: {kappa:.2f}")
    return acc, kappa

def plot_results(results):
    models = list(results.keys())
    accs = [results[m]["acc"] for m in models]
    kappas = [results[m]["kappa"] for m in models]

    x = range(len(models))
    plt.bar(x, accs, width=0.4, label="Accuracy", align="center")
    plt.bar([p + 0.4 for p in x], kappas, width=0.4, label="Kappa", align="center")
    plt.xticks([p + 0.2 for p in x], models)
    plt.ylabel("Score")
    plt.title("Model vs Human Agreement")
    plt.legend()
    plt.show()

# Display chunk-wise comparison table with conversation text
def display_chunk_comparison_with_text(prepared_chunks, human_chunks, llama_chunks, gemini_chunks):
    rows = []

    for idx, (chunk, h, l, g) in enumerate(zip(prepared_chunks, human_chunks, llama_chunks, gemini_chunks)):
        participant_texts = {}
        for participant in chunk['speaker'].unique():
            participant_turns = chunk[chunk['speaker'] == participant]['utterance'].tolist()
            participant_texts[participant] = " | ".join(participant_turns)

        for participant in h.keys():
            rows.append({
                "Chunk": idx+1,
                "Participant": participant,
                "Conversation": participant_texts.get(participant, ""),
                "Human": h[participant],
                "Llama": l.get(participant, "Neutral"),
                "Gemini": g.get(participant, "Neutral")
            })

    df_comp = pd.DataFrame(rows)
    print("\n=== Chunk-wise Comparison Table with Text ===")
    pd.set_option('display.max_colwidth', None)
    display(df_comp)
    return df_comp

# Plot trait distribution per participant (human + models)
def plot_trait_distribution(human_chunks, llama_chunks, gemini_chunks):
    participants = set()
    for chunk in human_chunks:
        participants.update(chunk.keys())
    participants = sorted(participants)

    fig, axes = plt.subplots(len(participants), 1, figsize=(8, 4*len(participants)))
    if len(participants) == 1:
        axes = [axes]

    for ax, participant in zip(axes, participants):
        # Collect trait counts
        human_counts = {}
        llama_counts = {}
        gemini_counts = {}

        for chunk in human_chunks:
            if participant in chunk:
                human_counts[chunk[participant]] = human_counts.get(chunk[participant], 0) + 1
        for chunk in llama_chunks:
            if participant in chunk:
                llama_counts[chunk[participant]] = llama_counts.get(chunk[participant], 0) + 1
        for chunk in gemini_chunks:
            if participant in chunk:
                gemini_counts[chunk[participant]] = gemini_counts.get(chunk[participant], 0) + 1

        all_traits = sorted(set(list(human_counts.keys()) + list(llama_counts.keys()) + list(gemini_counts.keys())))
        x = range(len(all_traits))
        ax.bar([p - 0.2 for p in x], [human_counts.get(tr,0) for tr in all_traits], width=0.2, label="Human")
        ax.bar(x, [llama_counts.get(tr,0) for tr in all_traits], width=0.2, label="Llama")
        ax.bar([p + 0.2 for p in x], [gemini_counts.get(tr,0) for tr in all_traits], width=0.2, label="Gemini")
        ax.set_xticks(x)
        ax.set_xticklabels(all_traits)
        ax.set_ylabel("Count")
        ax.set_title(f"Trait Distribution - {participant}")
        ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# 6. Run pipeline
DATASET_FILENAME = "human_chat.txt"
if not os.path.exists(DATASET_FILENAME):
    uploaded = files.upload()

df_raw = load_dataset(DATASET_FILENAME)
df_turns = preprocess_text(df_raw)

rounds_per_chunk = 4  # adjust as needed
chunk_size = rounds_per_chunk * 2
chunks = [df_turns.iloc[i:i+chunk_size] for i in range(0, len(df_turns), chunk_size)]

all_human_chunks = []
all_llama_chunks = []
all_gemini_chunks = []
prepared_chunks = []  # keep track for display

print("Trait explanations (refer to this before annotating):\n")
for trait, explanation in TRAIT_INFO.items():
    print(f"{trait}: {explanation}")
print("\n--- Starting annotation ---\n")

for idx, chunk in enumerate(chunks):
    print(f"\n--- Chunk {idx+1} ---\n")
    for _, row in chunk.iterrows():
        print(f"{row['speaker']}: {row['utterance']}")

    chunk_annotations = {}
    participants = chunk['speaker'].unique()
    for participant in participants:
        participant_turns = chunk[chunk['speaker'] == participant]['utterance'].tolist()
        print(f"\nParticipant: {participant}")
        print("Their turns in this chunk:")
        for turn in participant_turns:
            print(f"- {turn}")
        print("Available traits: " + " | ".join(TRAIT_INFO.keys()))
        choice = input("Enter trait for this participant in this chunk: ").strip().title()
        if choice not in TRAIT_INFO:
            choice = "Neutral"
        chunk_annotations[participant] = choice

    all_human_chunks.append(chunk_annotations)
    prepared_chunks.append(chunk)  # store for comparison display

    # Model annotation immediately
    llama_labels = model_annotation(chunk, model_name="llama")
    gemini_labels = model_annotation(chunk, model_name="gemini")
    all_llama_chunks.append(llama_labels)
    all_gemini_chunks.append(gemini_labels)

    # Stop condition
    if idx == len(chunks)-1:  # last chunk
        print("Reached the last chunk.")
        break
    next_chunk = input("Move to next chunk? (y/n): ").strip().lower()
    if next_chunk != "y":
        print("Stopping annotation as requested by user.")
        break

# Aggregate human traits
overall_traits, trait_distribution = aggregate_traits(all_human_chunks)

# Evaluate
results = {}
acc, kappa = evaluate_model(all_human_chunks, all_llama_chunks, "Llama")
results["Llama"] = {"acc": acc, "kappa": kappa}
acc, kappa = evaluate_model(all_human_chunks, all_gemini_chunks, "Gemini")
results["Gemini"] = {"acc": acc, "kappa": kappa}

plot_results(results)

# Display overall trait distribution
print("\n=== Overall Traits per Participant ===")
for participant, trait in overall_traits.items():
    print(f"{participant}: {trait}")

print("\n=== Trait Distribution by Participant ===")
for participant, counts in trait_distribution.items():
    print(f"{participant}: {counts}")

# Display chunk-wise comparison table
df_chunk_comparison = display_chunk_comparison_with_text(
    prepared_chunks, all_human_chunks, all_llama_chunks, all_gemini_chunks
)

# Plot trait distribution per participant (human + models)
plot_trait_distribution(all_human_chunks, all_llama_chunks, all_gemini_chunks)